In [2]:
info = """
LangChain 1.0 - Checkpointing (检查点持久化)
=========================================

本模块重点讲解：
1. SqliteSaver - SQLite 持久化（LangGraph 提供）
2. 与 InMemorySaver 的区别
3. 跨进程、跨重启的对话持久化
4. 实际应用场景

⚠️ API 说明：
- 本模块使用 `create_react_agent`（来自 langgraph.prebuilt）
- Checkpointing 是 LangGraph 的核心特性
- 配合 create_react_agent 可以完整展示状态持久化

工作流程：
  1. invoke 前：LangGraph 自动调用 checkpointer.get(thread_id="user_123")      
    - 查询数据库，读取该 thread_id 的历史消息
    - 如果是第一次，返回空列表
  2. invoke 中：Agent 处理时会看到完整历史
  state = {
      "messages": [历史消息1, 历史消息2, 新消息]  # 自动合并
  }
  3. invoke 后：LangGraph 自动调用 checkpointer.put(thread_id, state)
    - 将新的完整状态写入数据库
"""

print(info)


LangChain 1.0 - Checkpointing (检查点持久化)

本模块重点讲解：
1. SqliteSaver - SQLite 持久化（LangGraph 提供）
2. 与 InMemorySaver 的区别
3. 跨进程、跨重启的对话持久化
4. 实际应用场景

⚠️ API 说明：
- 本模块使用 `create_react_agent`（来自 langgraph.prebuilt）
- Checkpointing 是 LangGraph 的核心特性
- 配合 create_react_agent 可以完整展示状态持久化

工作流程：
  1. invoke 前：LangGraph 自动调用 checkpointer.get(thread_id="user_123")      
    - 查询数据库，读取该 thread_id 的历史消息
    - 如果是第一次，返回空列表
  2. invoke 中：Agent 处理时会看到完整历史
  state = {
      "messages": [历史消息1, 历史消息2, 新消息]  # 自动合并
  }
  3. invoke 后：LangGraph 自动调用 checkpointer.put(thread_id, state)
    - 将新的完整状态写入数据库



In [4]:
from langgraph.checkpoint.sqlite import SqliteSaver
import  os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

In [5]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

In [31]:
model = init_chat_model("groq:openai/gpt-oss-120b", api_key=GROQ_API_KEY)


In [7]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x110284440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1102852b0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
@tool
def get_order_status(order_id: str) -> str:
    """查询订单状态"""
    orders = {
        "12345": "已发货，预计明天送达",
        "67890": "配送中，今天下午送达"
    }
    return orders.get(order_id, "订单不存在")

In [10]:
print(
    """
    示例1：InMemorySaver 的限制

    问题：
    - 程序重启后丢失
    - 无法跨进程共享
    - 不适合生产环境
    """)


    示例1：InMemorySaver 的限制

    问题：
    - 程序重启后丢失
    - 无法跨进程共享
    - 不适合生产环境
    


In [11]:
print("\n" + "="*70)
print("示例 1：InMemorySaver 的限制")
print("="*70)


示例 1：InMemorySaver 的限制


In [12]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。记住用户告诉你的信息。",
    checkpointer=InMemorySaver()
)

config = {"configurable": {"thread_id": "user_123"}}

print("\n第一轮对话：")
agent.invoke(
    {"messages": [{"role": "user", "content": "我叫张三"}]},
    config=config
)
print("用户: 我叫张三")

print("\n第二轮对话（同一进程内）：")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫什么？"}]},
    config=config
)
print(f"Agent: {response['messages'][-1].content}")


第一轮对话：
用户: 我叫张三

第二轮对话（同一进程内）：
Agent: <think>
好的，用户之前告诉我他叫张三，现在问我“我叫什么？”，我需要确认他的名字是否正确。根据之前的对话记录，用户确实提到过“我叫张三”。所以我的回答应该是确认他的名字，并可能询问是否需要帮助。要确保回答准确无误，同时保持友好和帮助的态度。不需要添加额外信息，直接回答问题即可。
</think>

你叫张三。需要我帮忙确认其他信息吗？


In [13]:
print("\n限制：")
print("  ❌ 程序重启后，对话历史丢失")
print("  ❌ 无法在不同进程间共享")
print("  ❌ 不适合生产环境")
print("\n解决方案：使用 SqliteSaver 持久化到数据库")


限制：
  ❌ 程序重启后，对话历史丢失
  ❌ 无法在不同进程间共享
  ❌ 不适合生产环境

解决方案：使用 SqliteSaver 持久化到数据库


In [14]:
print("""
    示例2：使用 SqliteSaver 实现持久化

    关键：
    1. SqliteSaver.from_conn_string("sqlite:///path/to/db.sqlite")
    2. 对话持久化到 SQLite 文件
    3. 程序重启后仍可恢复
    """)


    示例2：使用 SqliteSaver 实现持久化

    关键：
    1. SqliteSaver.from_conn_string("sqlite:///path/to/db.sqlite")
    2. 对话持久化到 SQLite 文件
    3. 程序重启后仍可恢复
    


In [15]:
print("\n" + "="*70)
print("示例 2：SqliteSaver - 持久化到 SQLite")
print("="*70)


示例 2：SqliteSaver - 持久化到 SQLite


In [16]:
db_path = "checkpoints.sqlite"  # 相对路径

In [17]:
with SqliteSaver.from_conn_string(db_path) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[],
        system_prompt="你是一个有用的助手",
        checkpointer=checkpointer
    )
    config = {"configurable": {"thread_id": "persistent_user"}}
    print("\n第一轮对话：")
    agent.invoke(
        {"messages": [{"role": "user", "content": "我叫李四"}]},
        config=config
    )
    print("\n第二轮对话（同一进程内）：")
    response = agent.invoke(
        {"messages": [{"role": "user", "content": "我叫什么？"}]},
        config=config
    )
    print(f"Agent: {response['messages'][-1].content}")


第一轮对话：

第二轮对话（同一进程内）：
Agent: <think>
嗯，用户之前告诉我他叫李四，现在又问“我叫什么？”，可能是在测试我的记忆能力，或者想确认我是否记住了他的名字。我需要确认之前的对话记录，确保没有混淆。用户可能希望得到一个直接的回应，确认他的名字被正确记住。此外，用户可能有更深层的需求，比如想检查我的功能或确保对话的连贯性。我应该用友好的语气回答，同时保持简洁，避免冗长。需要确保回复准确，并且让用户感到被重视。
</think>

你叫李四！我记住了哦。有什么需要我帮忙的吗？😄


In [19]:
print(f"\n关键点：")
print(f"  ✅ 对话保存到文件：{db_path}")
print("  ✅ 程序重启后仍可恢复")
print("  ✅ 可以跨进程访问")
print("  ✅ 适合生产环境")


关键点：
  ✅ 对话保存到文件：checkpoints.sqlite
  ✅ 程序重启后仍可恢复
  ✅ 可以跨进程访问
  ✅ 适合生产环境


In [20]:
print(    """
    示例3：验证持久化效果

    模拟程序重启后，从数据库恢复对话
    """)


    示例3：验证持久化效果

    模拟程序重启后，从数据库恢复对话
    


In [21]:
print("\n" + "="*70)
print("示例 3：验证持久化（模拟重启后恢复）")
print("="*70)


示例 3：验证持久化（模拟重启后恢复）


In [22]:
db_path = "checkpoints.sqlite"

In [23]:
config = {"configurable": {"thread_id": "persistent_user"}}


In [24]:
print("\n[模拟程序重启...]")



[模拟程序重启...]


In [ ]:
with SqliteSaver.from_conn_string(db_path) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[],
        system_prompt="你是一个有帮助的助手。",
        checkpointer=checkpointer
    )
    print("\n第三轮对话（新进程，但 thread_id 相同）：")
    print("用户: 我之前说我叫什么？")
    response = agent.invoke(
        {"messages": [{"role": "user", "content": "我之前说我叫什么？"}]},
        config=config
    )
    print(f"Agent: {response['messages'][-1].content}")

    


第三轮对话（新进程，但 thread_id 相同）：
用户: 我之前说我叫什么？
Agent: <think>
好的，用户问：“我之前说我叫什么？”我需要回顾之前的对话记录。用户之前已经说过两次他叫李四，第一次是用户自己说的，然后我回复了。第二次用户再次确认名字，我再次回应。现在用户第三次问同样的问题，可能是想确认我是否记得，或者想测试我的记忆能力。

首先，用户可能是在测试我是否真的记得他的名字，或者是否有记忆功能。也有可能用户只是想确认之前的对话内容，确保没有误解。另外，用户可能在练习对话流程，看看系统的反应是否一致。

我需要确保我的回答准确，并且保持友好。应该直接回答他的名字，并且确认记得，同时保持开放式的提问，看看是否需要进一步帮助。同时，要注意不要重复太多，避免显得机械。可能需要加入一些表情符号来增加亲切感。

检查之前的对话：用户第一次说“我叫李四”，我回复了名字，并问有什么帮忙的。用户第二次问“我叫什么？”，我再次确认名字，并邀请提问。现在第三次问同样的问题，我需要再次确认名字，并保持友好的态度。可能用户有记忆障碍，或者只是想确保系统可靠。需要保持耐心和一致。

总结回答的结构：确认名字，表达记得，主动提供帮助，保持友好。确保回答简洁明了，避免冗长。可能用户希望系统能够记住对话历史，所以正确的回应是关键。
</think>

你之前告诉我你叫李四！我记住了~ 😊 还需要我帮你做点什么吗？

关键点：
  - Agent 记得之前的对话（李四）
  - 即使创建了新的 agent 实例
  - 因为 SQLite 保存了完整历史


In [27]:
print("\n关键点：")
print("  - Agent 记得之前的对话（李四）")
print("  - 即使创建了新的 agent 实例")
print("  - 因为 SQLite 保存了完整历史")


关键点：
  - Agent 记得之前的对话（李四）
  - 即使创建了新的 agent 实例
  - 因为 SQLite 保存了完整历史


In [28]:
print("""
    示例4：管理多个用户的持久化会话

    每个用户有独立的 thread_id
    """)


    示例4：管理多个用户的持久化会话

    每个用户有独立的 thread_id
    


In [29]:
print("\n" + "="*70)
print("示例 4：多用户会话管理")
print("="*70)



示例 4：多用户会话管理


In [32]:
db_path = "multi_user.sqlite"

with SqliteSaver.from_conn_string(db_path) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[],
        system_prompt="你是一个有帮助的助手。",
        checkpointer=checkpointer
    )

    # 用户 A
    print("\n[用户 A 的对话]")
    config_a = {"configurable": {"thread_id": "user_alice"}}
    agent.invoke(
        {"messages": [{"role": "user", "content": "我是 Alice，我喜欢编程"}]},
        config_a
    )
    print("Alice: 我是 Alice，我喜欢编程")

    # 用户 B
    print("\n[用户 B 的对话]")
    config_b = {"configurable": {"thread_id": "user_bob"}}
    agent.invoke(
        {"messages": [{"role": "user", "content": "我是 Bob，我喜欢设计"}]},
        config_b
    )
    print("Bob: 我是 Bob，我喜欢设计")

    # 回到用户 A
    print("\n[用户 A 继续对话]")
    response_a = agent.invoke(
        {"messages": [{"role": "user", "content": "我喜欢什么？"}]},
        config_a
    )
    print(f"Alice: 我喜欢什么？")
    print(f"Agent: {response_a['messages'][-1].content}")

    # 回到用户 B
    print("\n[用户 B 继续对话]")
    response_b = agent.invoke(
        {"messages": [{"role": "user", "content": "我喜欢什么？"}]},
        config_b
    )
    print(f"Bob: 我喜欢什么？")
    print(f"Agent: {response_b['messages'][-1].content}")


[用户 A 的对话]
Alice: 我是 Alice，我喜欢编程

[用户 B 的对话]
Bob: 我是 Bob，我喜欢设计

[用户 A 继续对话]
Alice: 我喜欢什么？
Agent: Alice，你已经告诉我两件事：

1. **你叫 Alice**  
2. **你喜欢编程**

如果要进一步探讨“你到底喜欢什么”，我们可以把“喜欢”拆成几个维度来思考，这样既能帮助你更清晰地认识自己的兴趣，也能为以后挑选学习方向或项目提供参考。

---

## 1️⃣ 你在编程里最享受的环节是什么？

| 编程环节 | 典型感受 | 可能对应的兴趣方向 |
|----------|----------|-------------------|
| **设计算法 / 解谜** | 解决难题时的“哇，这太酷了” | 算法竞赛、机器学习、数据分析 |
| **搭建界面 / 交互** | 看到页面/APP动起来的成就感 | 前端、移动开发、UI/UX |
| **搭建后端系统** | 让数据流动、业务顺畅的满足感 | 后端、云服务、DevOps |
| **优化性能** | 代码跑得更快、更省资源的快感 | 系统架构、性能调优、嵌入式 |
| **写工具 / 脚本** | 自动化重复工作、帮别人省事的愉悦 | 自动化、DevOps、开源工具 |
| **协作 & 贡献** | 与团队/社区一起完成大事的成就感 | 开源项目、团队开发、技术博客 |

> **思考小练习**：回想最近一次写代码时，你最开心的瞬间是哪一步？把它标记在上表里对应的列上，这能帮助你定位最核心的兴趣点。

---

## 2️⃣ 你更倾向于哪类“产出”？

- **产品/应用**（比如一个网站、App、小游戏） → 适合全栈或特定前/后端方向。  
- **数据洞察**（报告、可视化、模型） → 适合数据科学/机器学习。  
- **系统/工具**（库、脚本、自动化） → 适合底层开发或 DevOps。  
- **游戏/交互体验** → 适合游戏开发或交互式媒体。

---

## 3️⃣ 试试看：**快速兴趣实验**  

挑选 **两天**，各花 **1–2 小时** 完成以下任意两个小项目（任选其一即可）：

1. **前端**：用 HTML+CSS+JavaScript 做一个「待办清单」页

In [33]:
"""
示例5：工具调用 + 持久化

Agent 记住工具调用历史
"""
print("\n" + "="*70)
print("示例 5：工具调用 + 持久化")
print("="*70)


示例 5：工具调用 + 持久化


In [34]:
db_path = "tools.sqlite"

with SqliteSaver.from_conn_string(db_path) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[get_order_status],
        system_prompt="你是一个有帮助的助手。",
        checkpointer=checkpointer
    )

    config = {"configurable": {"thread_id": "customer_001"}}

    print("\n第一轮：查询订单")
    print("客户: 查询订单 12345 的状态")
    response1 = agent.invoke(
        {"messages": [{"role": "user", "content": "查询订单 12345 的状态"}]},
        config=config
    )
    print(f"Agent: {response1['messages'][-1].content}")

    print("\n第二轮：询问之前的查询结果")
    print("客户: 我的订单什么时候到？")
    response2 = agent.invoke(
        {"messages": [{"role": "user", "content": "我的订单什么时候到？"}]},
        config=config
    )
    print(f"Agent: {response2['messages'][-1].content}")


第一轮：查询订单
客户: 查询订单 12345 的状态
Agent: 订单 12345 的当前状态是**已发货**，预计将在**明天送达**。祝您使用愉快！如果还有其他问题，随时告诉我。

第二轮：询问之前的查询结果
客户: 我的订单什么时候到？
Agent: 根据我们刚刚查询的结果，您的订单 12345 已经发货，预计将在**明天**送达。如果您在明天仍未收到，请随时告诉我，我会帮您进一步跟进。祝您收货愉快！


In [35]:
print("\n关键点：")
print("  - Agent 记住了订单 12345 的查询结果")
print("  - 工具调用历史也被持久化")
print("  - 无需重复调用工具")


关键点：
  - Agent 记住了订单 12345 的查询结果
  - 工具调用历史也被持久化
  - 无需重复调用工具


In [36]:
"""
    示例6：实际应用 - 持久化客服系统

    场景：客户可能分多次咨询，需要记住历史
    """

'\n    示例6：实际应用 - 持久化客服系统\n\n    场景：客户可能分多次咨询，需要记住历史\n    '

In [37]:
print("\n" + "="*70)
print("示例 6：实际应用 - 持久化客服系统")
print("="*70)

db_path = "customer_service.sqlite"


示例 6：实际应用 - 持久化客服系统


In [38]:
db_path

'customer_service.sqlite'

In [41]:
with SqliteSaver.from_conn_string(db_path) as checkpointer:
    agent = create_agent(
        model=model,
        tools=[get_order_status],
        checkpointer=checkpointer,
        system_prompt="""你是客服助手。
特点：
- 记住客户之前的咨询
- 友好、耐心
- 使用工具查询订单""",)
    
    customer_id = "customer_001"
    config = {"configurable": {"thread_id": customer_id}}
    print("\n客户首次咨询：")
    conversations_morning = [
            "你好，我想查询订单",
            "订单号是 12345"
        ]
    for msg in conversations_morning:
        print(f"客户: {msg}")
        response = agent.invoke(
            {"messages": [{"role": "user", "content": msg}]},
            config=config
        )
        print(f"Agent: {response['messages'][-1].content}")

    print("\n客户下午再次咨询：")
    conversations_afternoon = [
            "我的订单什么时候到？",
            "能查一下吗？"
        ]
    for msg in conversations_afternoon:
        print(f"客户: {msg}")
        response = agent.invoke(
            {"messages": [{"role": "user", "content": msg}]},
            config=config
        )
        print(f"Agent: {response['messages'][-1].content}")


客户首次咨询：
客户: 你好，我想查询订单
Agent: 您好！很高兴为您服务 😊  
请您提供一下订单号（order_id），我这边就可以帮您查询订单的最新状态啦。
客户: 订单号是 12345
Agent: 好的，已为您查询到订单 **12345** 的状态：

- **已发货**  
- **预计送达时间：明天**  

如果还有其他需要帮助的，随时告诉我哦！祝您生活愉快！

客户下午再次咨询：
客户: 我的订单什么时候到？
Agent: 您的订单 **12345** 预计将在 **明天** 送达。  
如果您还有其他问题或需要进一步的帮助，请随时告诉我！祝您有个愉快的一天 😊
客户: 能查一下吗？
Agent: 已为您再次查询到订单 **12345** 的最新状态：

- **已发货**  
- **预计送达时间：明天**  

如有其他疑问或需要进一步协助，随时告诉我！祝您生活愉快 😊


In [42]:
print("\n" + "="*70)
print("示例 7：SqliteSaver 参数详解")
print("="*70)

print("""
SqliteSaver 创建方式：

1. from_conn_string + with 语句（推荐）
with SqliteSaver.from_conn_string("checkpoints.sqlite") as checkpointer:
    system_prompt="你是一个有帮助的助手。"
    agent = create_agent(model=model, checkpointer=checkpointer)
    agent.invoke(...)

- 自动管理连接和资源
- 支持相对路径和绝对路径
- 最简单安全的方式
- 确保正确释放数据库连接
- 注意：直接传文件路径，不要加 sqlite:/// 前缀

2. 使用 sqlite3.connect（高级）
import sqlite3
conn = sqlite3.connect("checkpoints.sqlite")
checkpointer = SqliteSaver(conn)

- 需要手动管理连接
- 适合需要自定义连接参数的场景

数据库文件路径：
- 相对路径：checkpoints.sqlite（当前目录）
- 绝对路径：C:/Users/xxx/data/checkpoints.sqlite（Windows）
- 内存数据库：:memory:（测试用，程序退出即丢失）

最佳实践：
✅ 始终使用 with 语句管理 SqliteSaver
✅ 直接传文件路径，不要加 sqlite:/// 前缀
✅ 生产环境：使用绝对路径
✅ 开发测试：使用相对路径
✅ 单元测试：使用 :memory:
✅ 定期备份数据库文件
""")


示例 7：SqliteSaver 参数详解

SqliteSaver 创建方式：

1. from_conn_string + with 语句（推荐）
with SqliteSaver.from_conn_string("checkpoints.sqlite") as checkpointer:
    system_prompt="你是一个有帮助的助手。"
    agent = create_agent(model=model, checkpointer=checkpointer)
    agent.invoke(...)

- 自动管理连接和资源
- 支持相对路径和绝对路径
- 最简单安全的方式
- 确保正确释放数据库连接
- 注意：直接传文件路径，不要加 sqlite:/// 前缀

2. 使用 sqlite3.connect（高级）
import sqlite3
conn = sqlite3.connect("checkpoints.sqlite")
checkpointer = SqliteSaver(conn)

- 需要手动管理连接
- 适合需要自定义连接参数的场景

数据库文件路径：
- 相对路径：checkpoints.sqlite（当前目录）
- 绝对路径：C:/Users/xxx/data/checkpoints.sqlite（Windows）
- 内存数据库：:memory:（测试用，程序退出即丢失）

最佳实践：
✅ 始终使用 with 语句管理 SqliteSaver
✅ 直接传文件路径，不要加 sqlite:/// 前缀
✅ 生产环境：使用绝对路径
✅ 开发测试：使用相对路径
✅ 单元测试：使用 :memory:
✅ 定期备份数据库文件

